In [1645]:
from FlexivPy.joy import XboxController
controller = XboxController()

In [1646]:
controller.calibrate()

In [1680]:
controller.getAllStates()

{'left_joy': array([ 0.02359009, -0.04129028]),
 'right_joy': array([0., 0.]),
 'triggers': array([0., 0.]),
 'A': 0,
 'B': 0,
 'X': 0,
 'Y': 0,
 'bumbers': (0, 0)}